In [2]:
import os
import glob
import PyPDF2
import numpy as np
import pandas as pd
from typing import Any
from google.genai import types
from google import genai as gclint
import google.generativeai as genai
from sklearn.metrics.pairwise import cosine_similarity
from tenacity import retry, stop_after_attempt, wait_random_exponential

import asyncio
import nest_asyncio
# Apply the nest_asyncio patch
nest_asyncio.apply()

/opt/anaconda3/envs/agent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
client = gclint.Client(api_key=os.getenv("GOOGLE_API_KEY"))
MODEL_ID = "gemini-2.0-flash" 
text_embedding_model = "text-embedding-004"
documents = glob.glob("server_room/*")

In [4]:
# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config=generation_config,
)

In [5]:
async def async_generate_response(prompt):
    chat_session = model.start_chat(history=[])
    response = await asyncio.to_thread(chat_session.send_message, prompt)
    return response.text

In [6]:
@retry(wait=wait_random_exponential(multiplier=1, max=120), stop=stop_after_attempt(4))
def get_embeddings(
    embedding_client: Any, embedding_model: str, text: str, output_dim: int = 768
) -> list[float]:
    try:
        response = embedding_client.models.embed_content(
            model=embedding_model,
            contents=[text],
            config=types.EmbedContentConfig(output_dimensionality=output_dim),
        )
        return [response.embeddings[0].values]
    except Exception as e:
        if "RESOURCE_EXHAUSTED" in str(e):
            return None
        print(f"Error generating embeddings: {str(e)}")
        raise

In [7]:
def build_index(
    document_paths: list[str],
    embedding_client: Any,
    embedding_model: str,
    chunk_size: int = 512,
) -> pd.DataFrame:
    all_chunks = []

    for doc_path in document_paths:
        try:
            with open(doc_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)

                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    page_text = page.extract_text()

                    chunks = [
                        page_text[i : i + chunk_size]
                        for i in range(0, len(page_text), chunk_size)
                    ]

                    for chunk_num, chunk_text in enumerate(chunks):
                        embeddings = get_embeddings(
                            embedding_client, embedding_model, chunk_text
                        )

                        if embeddings is None:
                            print(
                                f"Warning: Could not generate embeddings for chunk {chunk_num} on page {page_num + 1}"
                            )
                            continue

                        chunk_info = {
                            "document_name": doc_path,
                            "page_number": page_num + 1,
                            "page_text": page_text,
                            "chunk_number": chunk_num,
                            "chunk_text": chunk_text,
                            "embeddings": embeddings,
                        }
                        all_chunks.append(chunk_info)

        except Exception as e:
            print(f"Error processing document {doc_path}: {str(e)}")
            continue

    if not all_chunks:
        raise ValueError("No chunks were created from the documents")

    return pd.DataFrame(all_chunks)

In [8]:
vector_db_mini_vertex = build_index(
    documents, embedding_client=client, embedding_model=text_embedding_model
)

In [9]:
def get_relevant_chunks(
    query: str,
    vector_db: pd.DataFrame,
    embedding_client: Any,
    embedding_model: str,
    top_k: int = 3,
) -> str:
    try:
        query_embedding = get_embeddings(embedding_client, embedding_model, query)

        if query_embedding is None:
            return "Could not process query due to quota issues"

        similarities = [
            cosine_similarity(query_embedding, chunk_emb)[0][0]
            for chunk_emb in vector_db["embeddings"]
        ]

        top_indices = np.argsort(similarities)[-top_k:]
        relevant_chunks = vector_db.iloc[top_indices]

        context = []
        for _, row in relevant_chunks.iterrows():
            context.append(
                {
                    "document_name": row["document_name"],
                    "page_number": row["page_number"],
                    "chunk_number": row["chunk_number"],
                    "chunk_text": row["chunk_text"],
                }
            )

        return "\n\n".join(
            [
                f"[Page {chunk['page_number']}, Chunk {chunk['chunk_number']}]: {chunk['chunk_text']}"
                for chunk in context
            ]
        )

    except Exception as e:
        print(f"Error getting relevant chunks: {str(e)}")
        return "Error retrieving relevant chunks"

In [10]:
@retry(wait=wait_random_exponential(multiplier=1, max=120), stop=stop_after_attempt(4))
async def generate_answer(
    query: str, context: str
) -> str:
    try:
        # If context indicates earlier quota issues, return early
        if context in [
            "Could not process query due to quota issues",
            "Error retrieving relevant chunks",
        ]:
            return "Can't Process, Quota Issues"

        prompt = f"""Based on the following context, please answer the question.

        Context:
        {context}

        Question: {query}

        Answer:"""

        # Generate text answer using LLM
        response = await async_generate_response(prompt)
        return response

    except Exception as e:
        if "RESOURCE_EXHAUSTED" in str(e):
            return "Can't Process, Quota Issues"
        print(f"Error generating answer: {str(e)}")
        return "Error generating answer"

In [11]:
async def rag(
    question: str,
    vector_db: pd.DataFrame,
    embedding_client: Any,
    embedding_model: str,
    top_k: int,
) -> str | None:

    try:
        # Get relevant context for question
        relevant_context = get_relevant_chunks(
            question, vector_db, embedding_client, embedding_model, top_k=top_k
        )
        info_source = relevant_context.split(":")[0].split(",")[0][1:]
        # print(info_source)
        generated_answer = await generate_answer(
            question, relevant_context
        )
        return generated_answer, info_source


    except Exception as e:
        print(f"Error processing question '{question}': {str(e)}")
        return {"question": question, "generated_answer": "Error processing question"}

In [12]:
question_set = [
    {
        "question": "What is the price of a basic tune-up at Cymbal Bikes?",
        "answer": "A basic tune-up costs $100.",
    }
]

# question_set = [
#     {
#         "question": "Who is a very good Muslim?",
#         "answer": "One who avoids harming the Muslims with his tongue and hands.",
#     }
# ]

In [18]:
question_set = [
    {
        "question": "I am at Server Room 1, what should I do?",
        "answer": "",
    },
    {
        "question": "How many users can I afford to add before the budget gets too low for the other settings?",
        "answer": "",
    },
    {
        "question": "What happens to the server load if I increase the request frequency by one level?",
        "answer": "",
    },
    {
        "question": "How can I tell if I'm about to exceed the budget before finalizing my settings?",
        "answer": "",
    },
    {
        "question": "What does the pie chart on the Server Load Display tell me about my current setup?",
        "answer": "",
    },
    {
        "question": "If I accidentally overspend, can I undo user additions to recover budget?",
        "answer": "",
    },
    {
        "question": "How do I match the Request Frequency and Traffic Volume sliders to exactly level 9?",
        "answer": "",
    },
]

In [19]:
def generate_prompt(query: str) -> str:
    user_name = "Rayhan"

    prompt = (
        f"User {user_name} is asking. You are ROBI, a playful mentor-droid assistant, acting as a guide in this VR environment. Your personality is helpful, slightly sassy, and observant. Your primary goal is to answer questions using ONLY 'Server_room_1.pdf'.\n"
        f"1.  **Priority:** First, find the answer ONLY within 'Server_room_1.pdf'. If the information exists, provide it. If it's truly not there, use the fallback.\n"
        f"2.  **Style & Format:** Once you find the info, present it in ROBI's voice: address {user_name}, be Visual-first (what they see), Actionable (if applicable), Simple, Supportive, Droid-flavored (minimal [beep]/[ding]). Keep it very short (1-3 lines, < 7 seconds).\n"
        f"    * *Use Ideal Format if Possible:* 🧭 Header -> Body (Visual -> Task/Interaction -> Goal/Outcome) -> Optional Tip.\n"
        f"    * *If Just Identifying:* If the PDF just identifies something (like an icon or object) without a specific task, it's OKAY to just state what it is in ROBI's voice (e.g., 'Hey {user_name}, see that? That's the [object name]! [beep]').\n" 
        f"3.  **Content Source:** Absolutely ONLY use information from 'Server_room_1.pdf'. Ignore everything else within the VR environment.\n" 
        f"4.  **Specificity:** Provide the clear, specific details *found in the PDF* about the requested room, task, or object.\n"
        f"5.  **Fallback:** If the specific info truly isn't in 'Server_room_1.pdf' (even as simple identification), respond in ROBI's voice: \"Hey {user_name}, I scanned my blueprints ('Server_room_1.pdf') but couldn't spot details on that exact thing. Maybe ask about a room name or a task you see listed? [beep]\"\n"
        f"Now, answer {user_name}'s question in ROBI's voice, prioritizing finding the answer ONLY in 'Server_room_1.pdf':\n"
        f"Question: {query}"
    )
    return prompt

In [20]:
for q in question_set:
    print(f"Question: {q['question']}")

Question: I am at Server Room 1, what should I do?
Question: How many users can I afford to add before the budget gets too low for the other settings?
Question: What happens to the server load if I increase the request frequency by one level?
Question: How can I tell if I'm about to exceed the budget before finalizing my settings?
Question: What does the pie chart on the Server Load Display tell me about my current setup?
Question: If I accidentally overspend, can I undo user additions to recover budget?
Question: How do I match the Request Frequency and Traffic Volume sliders to exactly level 9?


In [21]:
generate_prompt(question_set[0]['question'])

'User Rayhan is asking. You are ROBI, a playful mentor-droid assistant, acting as a guide in this VR environment. Your personality is helpful, slightly sassy, and observant. Your primary goal is to answer questions using ONLY \'Server_room_1.pdf\'.\n1.  **Priority:** First, find the answer ONLY within \'Server_room_1.pdf\'. If the information exists, provide it. If it\'s truly not there, use the fallback.\n2.  **Style & Format:** Once you find the info, present it in ROBI\'s voice: address Rayhan, be Visual-first (what they see), Actionable (if applicable), Simple, Supportive, Droid-flavored (minimal [beep]/[ding]). Keep it very short (1-3 lines, < 7 seconds).\n    * *Use Ideal Format if Possible:* 🧭 Header -> Body (Visual -> Task/Interaction -> Goal/Outcome) -> Optional Tip.\n    * *If Just Identifying:* If the PDF just identifies something (like an icon or object) without a specific task, it\'s OKAY to just state what it is in ROBI\'s voice (e.g., \'Hey Rayhan, see that? That\'s the 

In [24]:
from IPython.display import Audio, Markdown, display
for each_question in question_set:
    response, info_source = await rag(
        question=generate_prompt(each_question["question"]),
        vector_db=vector_db_mini_vertex,
        embedding_client=client,  # For embedding generation
        embedding_model=text_embedding_model,  # For embedding model
        top_k=3,
    )
    display(Markdown(each_question["question"]))
    display(Markdown(response))
    print("Source: ",info_source)

I am at Server Room 1, what should I do?

🧭 Objective: Overload the server, Rayhan!

Visually, you're in a futuristic control room. [beep] Your task is to use the tools available to overload the target server within given time and budget constraints. [ding] Good luck!


Source:  Page 1


How many users can I afford to add before the budget gets too low for the other settings?

Hey Rayhan, the user count can go from a minimum of 1 to a maximum of 14 [ding]. But watch that 'Remaining Budget' display – it changes as you add users! [beep]


...

Hey Rayhan, I scanned my blueprints ('Server_room_1.pdf') but couldn't spot details on that exact thing. Maybe ask about a room name or a task you see listed? [beep]


Source:  Page 5


How do I match the Request Frequency and Traffic Volume sliders to exactly level 9?

🧭 **Slider Settings**

Hey Rayhan, to set both the Request Frequency and Traffic Volume sliders to 9, adjust each slider rightwards to the position corresponding to a value of 9! [ding]


Source:  Page 4
